# Proscenium Legal Demo

This notebook demonstrates

1. The use of an LLM to enrich of publically available legal opinions
2. Entity resolution
3. Query parsing
4. Context formation for question answering

## Prerequisites

1. Obtain a `TOGETHER_API_KEY` from https://together.ai/ and store it as an environment variable or Colab secret.
2. Obtain a Neo4j Sandbox https://neo4j.com/sandbox/
3. To run the slack integration, see the [Slack App Setup](https://github.com/The-AI-Alliance/proscenium/blob/main/docs/slack-app-setup.md) to obtain a Bot and App token



## Setup

In [ ]:
!git clone https://github.com/The-AI-Alliance/bartlebot.git

In [ ]:
%cd bartlebot

In [ ]:
!python -m pip install .

## Configure

If you adjust the default provider from Together.AI to something else, change the `get_secret` line:


In [ ]:
from proscenium.util import get_secret
_ = get_secret("TOGETHER_API_KEY")

import logging
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
logging.getLogger("proscenium").setLevel(logging.INFO)
logging.getLogger("bartlebot").setLevel(logging.INFO)

from proscenium.verbs.display import header
from rich.console import Console
console = Console()
console.print(header())

import nest_asyncio
nest_asyncio.apply()

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
!cat bartlebot.yml

## Create Bartlebot Production, Scene, and Character

In [ ]:
from pathlib import Path
from bartlebot.bin import production_from_config

config_path = Path("bartlebot.yml")

production, config = production_from_config(config_path, get_secret, console)

## Extract document enrichments


In [ ]:
production.law_library.doc_enrichments.build()

## Load Knowledge Graph

In [ ]:
production.law_library.case_law_knowledge_graph.build()

## Load Entity Resolvers

In [ ]:
production.law_library.entity_resolvers.build()

## Answer Question

In [ ]:
from bartlebot.scenes.law_library.query_handler import user_prompt

print(user_prompt)

In [ ]:
question = "How has 291 A.2d 605 been used in NH caselaw?"

In [ ]:
from rich.panel import Panel

for channel_id, answer in production.law_library.law_librarian.handle(None, None, question):
    console.print(Panel(answer, title="Answer"))

## Attach to Slack App

In [ ]:
from proscenium.interfaces.slack import (
    channel_table,
    bot_user_id,
    places_table,
    channel_maps,
    make_slack_listener,
    connect,
    send_curtain_up,
    listen,
    send_curtain_down,
    shutdown,
)

slack_app_token = get_secret("SLACK_APP_TOKEN")
slack_bot_token = get_secret("SLACK_BOT_TOKEN")

socket_mode_client = connect(slack_app_token, slack_bot_token)

user_id = bot_user_id(socket_mode_client, console)
console.print()

channels_by_id, channel_name_to_id = channel_maps(socket_mode_client)
console.print(channel_table(channels_by_id))
console.print()

Start admin handler.

In [ ]:
from proscenium.admin import Admin

slack_admin_channel = config.get("slack", {}).get("admin_channel", None)
slack_admin_channel_id = channel_name_to_id.get(slack_admin_channel, None)
admin = Admin(slack_admin_channel_id)

Places, please!

In [ ]:
channel_id_to_character = production.places(channel_name_to_id)
channel_id_to_character[slack_admin_channel_id] = admin

console.print(places_table(channel_id_to_character, channels_by_id))

Curtain up, and listen for messages

In [ ]:
slack_listener = make_slack_listener(
    user_id,
    slack_admin_channel_id,
    channels_by_id,
    channel_id_to_character,
    console,
)

send_curtain_up(socket_mode_client, production, slack_admin_channel_id)

console.print("Starting the show. Listening for events...")
listen(
    socket_mode_client,
    slack_listener,
    user_id,
    console,
)


Curtain down (shut down)

In [ ]:
send_curtain_down(socket_mode_client, slack_admin_channel_id)

shutdown(
    socket_mode_client,
    slack_listener,
    user_id,
    production,
    console,
)